### Core

In [ ]:
# Headers
import torch
from torch import nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
device = "cuda" if torch.cuda.is_available() else "cpu"
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.device_count())


In [ ]:
# Functions
def tensorprint(tensor):
    print("Shape: " , tensor.shape, " , Dimension: ", tensor.ndim , " \nDtype: ", tensor.dtype, " , Device: ", tensor.device)
    print("Max: ", tensor.amax(),f'[{tensor.argmax()}]', " , Min: ", tensor.amin(),f'[{tensor.argmin()}]')
    print(tensor ,'\n')

def plot_linear_predictions(train_data, train_label, 
                    test_data, test_labels, 
                    predictions = None):
                    plt.figure(figsize=(8,4))
                    plt.scatter(train_data,train_label, c="g", s=2, label="Training data")
                    plt.scatter(test_data, test_labels, c="r", s=2, label ="Testing data")
                    if predictions != None:
                        plt.scatter(test_data, predictions, c="b", s=2, label = "Predictions" )
                    plt.legend(prop= {"size": 10})

def plot_curves(epoch_count, loss_values, test_lost_values):
    plt.plot(epoch_count, loss_values, label="Train loss")
    plt.plot(epoch_count,test_lost_values, label="Test loss")
    plt.title("Training and test loss curves")
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
                    

### Tensor Fundamentals

In [ ]:
# scalar , vector, matrix
# scalar
scalar = torch.tensor(69)
tensorprint(scalar)

# vector
vector = torch.tensor([6,9])
tensorprint(vector)

# matrix
matrix = torch.tensor([ [2,4],
                        [5,9]])
tensorprint(matrix)

In [ ]:
# creating tensor
tensor = torch.tensor([[[1,2,3],
                        [4,5,6],
                        [7,8,9]]],
                        dtype = torch.float32,
                        device = "cpu",
                        requires_grad=False)
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)          # reproduce seed once
tensor_A = torch.rand(1,3,3)
tensor_B = torch.rand(1,3,3)
tensor_gA = tensor_A.to(device)         # change to gpu if available

int32_tensor = tensor.type(torch.int32)
random_image_tensor = torch.rand(size=(224,224,3))
range_zeros = torch.zeros_like(random_image_tensor)
zero_tensor = torch.zeros(3,3,5)
one_tensor = torch.ones(3,3,5)
permuted_tensor = tensor_A.permute(1,2,0)
range_tensor = torch.arange(start = 0, end = 15, step = 1)
numpy_a = np.arange(1.0,10.0)
tensor_nA = torch.from_numpy(numpy_a).type(torch.float32)
numpy_A = tensor_A.numpy()

# tensorprint(tensor)
# tensorprint(tensor_A)
# tensorprint(tensor_B)
tensorprint(permuted_tensor)

In [ ]:
# manipulating tensors
basic_operations_tn = torch.add(torch.mul(tensor, 11), 2)
element_mul_tn = torch.mul(tensor, basic_operations_tn)
tensor_B_transpose = tensor_B.mT
tensor_A_type_change = tensor_A.type(torch.int32)
tensor_A_reshaped = tensor.reshape(1,9)                 # not same memory as tensor_A
tensor_A_view = tensor_A.view(9,1)                      # same memory as tensor_A
tensor_A_stack = torch.stack([tensor_A, tensor],dim=1)  # stack tensor to each other hstack/vstack
tensor_A_squeeze = torch.squeeze(tensor_A_reshaped)     # remove single dimension [1,1,9] = [9]
tensor_A_unsqueeze = torch.unsqueeze(tensor_A_squeeze, dim=0)
tensor_A_permute = torch.permute(tensor_A, (2,1,0))

# tensor aggregation (min,max,sum, etc)
tensor_Amax = tensor_A.max()
tensor_Amin = tensor_A.min()
tensor_Asum = torch.sum(tensor_A)
tensor_Amean = torch.mean(tensor_A)
tensor_argmin = tensor_A.argmin()
tensor_argmax = tensor_A.argmax()

# indexing
tensor_index = [tensor[:, 0], tensor[:, 1], tensor[:,:,0], 
                tensor[:,:,1], tensor[:,1,1], 
                ]
for i in tensor_index:
    tensorprint(i)
# matrix multiplication = xrow * ycolumn 
matrix_matmul_tn = torch.matmul(tensor, tensor_A)
tensorprint(tensor_A)

### Pytorch Workflow

In [ ]:
# Creating parameters and data
weight = 0.9
bias = 0.4  
start = 0
end = 1
step = 0.01

X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias
len(X), len(y)

In [ ]:
# Creating train/test split
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

X_train, y_train = X_train.to(device) , y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)
plot_linear_predictions(X_train.cpu(),y_train.cpu(),X_test.cpu(),y_test.cpu())

In [ ]:
# Build model
class LinearRegressionModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        # Initialize model parameters using built in pytorch
        self.linear_layer = nn.Linear(in_features=1, out_features=1)
    
        # Initialize model parameters YOURSELF
        # self.weights = nn.Parameter(torch.randn(1,requires_grad= True, 
        #                                         dtype=torch.float))
        # self.bias = nn.Parameter(torch.randn(1,requires_grad = True,
        #                                         dtype=torch.float))

    
    # forward() defines computation in model
    def forward(self, x: torch.Tensor):
        return self.linear_layer(x)

        # computation when you initialize YOURSELF
        # return self.weights * x + self.bias

torch.manual_seed(69)
model_0 = LinearRegressionModel()
model_0.to(device)
parameters = model_0.state_dict()
with torch.inference_mode():                          # disables grad
    y_preds_wt= model_0(X_test)
plot_linear_predictions(X_train.cpu(),y_train.cpu(),X_test.cpu(),y_test.cpu(), y_preds_wt.cpu())

In [ ]:
# Training a model
# Loss functions & Optimizer
torch.manual_seed(69)
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(params = model_0.parameters(),
                            lr= 0.001)

epochs= 1000
epoch_count = []
loss_values = []
test_lost_values = []

for epoch in range(epochs):
    model_0.train()                             # train mode
    y_pred = model_0(X_train)                   # make predictions

    loss = loss_fn(y_pred, y_train)             # calculate loss w/ pred & train
    optimizer.zero_grad()                       # set to 0 the acuumulate
    loss.backward()                             # backpropogation
    optimizer.step()                            # gradient descent +1 accumulate

    model_0.eval()                              # tunrs settings not needed
    if epoch % 200 == 0:
        with torch.inference_mode():                          
            y_prednew = model_0(X_test)
            test_lost = loss_fn(y_prednew, y_test)
            plot_linear_predictions(X_train.cpu(),y_train.cpu(),X_test.cpu(),y_test.cpu(),y_prednew.cpu())    
        epoch_count.append(epoch)
        loss_values.append(loss.cpu())
        test_lost_values.append(test_lost.cpu())

converted_lossnp = np.array(torch.tensor(loss_values).numpy())                 

print(loss)
print(weight, bias, '\n', parameters)
plot_curves(epoch_count,converted_lossnp,test_lost_values)

In [ ]:
# Saving model
MODEL_PATH = Path("Models")
MODEL_PATH.mkdir(parents= True, exist_ok =True)
MODEL_NAME = "01_LinearRegression_0.pt"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

torch.save(model_0.state_dict(), MODEL_SAVE_PATH)

In [ ]:
# Loading model 
lmodel_0 = LinearRegressionModel()
lmodel_0.to(device)
lmodel_0.load_state_dict(torch.load(MODEL_SAVE_PATH))
with torch.inference_mode():
    lmodel_0_preds = lmodel_0(X_test)